In [61]:
import cv2
import pykep
import numpy as np
import imutils
import math
import pandas as pd  # for debug

Image tools

In [4]:
def debris_in_pixels(physical_size, vec, AOV=np.radians(39.23), dimensions = (1080, 1980)):
    """
    Returns how many pixels the object at a given distance should take up. Width, height
    :param physical_size: The size of the object
    :param alpha: diagonal angle of view
    :param dimensions: the pixel dimensions of the observer camera used
    :param vec: relative vector from observer to target in body CS.
    """
    pixels_per_length = np.linalg.norm(dimensions)/(np.tan(AOV)*vec[1])
    
    return math.ceil(physical_size[0]*pixels_per_length), math.ceil(physical_size[1]*pixels_per_length)


def resize(original_image, physical_size, vec):
    """
    Resize the image based on the physical size of the target object and the relative distance between 
    target object and observer
    :param physical_size: The size of the object in km
    :param distance: Distance between the target and observer planes
    """
    width, height = debris_in_pixels(physical_size, vec)
    if width == 0 and height ==0:
        return False  # can't resize, object too small
    else:
        return cv2.resize(original_image, (width, height), interpolation=cv2.INTER_AREA)

In [5]:
def to_pixels(r, shape,cols=1980, rows=1080, AOV=np.radians(39.23)):
    """
    Converts physical location to pixels and pixel coordinates. Returns rows, columns
    r is in body CS.
    """
    distance_to_screen = (rows**2+cols**2)**0.5/(2*np.tan(AOV))
    unit_r = r/np.linalg.norm(r)
    #print("unit vector", unit_r)
    k = distance_to_screen/unit_r[1]
    r_in_pixels = unit_r*k  # x,y,z --> cols,dist,rows
    print("location vector in pixels",r_in_pixels)
    
    return math.ceil(r_in_pixels[0]+cols/2-shape[1]/2),math.ceil(distance_to_screen),math.ceil(-r_in_pixels[2]+rows/2-shape[0]/2)

def get_left_corner(r, shape):
    """
    Return left corner in rows,cols
    """
    vec = to_pixels(r, shape)
    return (vec[2],vec[0])

In [6]:
def roi_in_frame(left_corner, object_size, frame):
    """
    Return the part of the ROI that is in the frame.
    :left_corner: the coordinates of the left corner of the object in px.
    :object_size: size of the object in pixels in CV2 coordinates (columns, rows)
    """
    frame_size = frame.shape # returns rows,cols
    ROI = [[left_corner[0], left_corner[0] + object_size[0]],[left_corner[1], left_corner[1] + object_size[1]]]
    
    if left_corner[1] + object_size[1] > frame_size[1]:
        print("outside the right bounding edge")
        # outside the right bounding edge
        ROI[1] = [left_corner[1], frame_size[1]]
    if left_corner[1] < 0:
        print("outside the left bounding edge")
        # outside the left bounding edge
        ROI[1] = [0, left_corner[1] + object_size[1]]

    if left_corner[0] + object_size[0] > frame_size[0]:
        print("outside the bottom bounding edge")
        # outside the bottom bounding edge
        ROI[0] = [left_corner[0], frame_size[0]]
    if left_corner[0] < 0:
        print("outside the upper bounding edge")
        # outside the upper bounding edge
        ROI[0] = [0, left_corner[0] + object_size[0]]

    return frame[ROI[0][0]:ROI[0][1], ROI[1][0]:ROI[1][1]]


def obj_in_frame(left_corner, frame, img):
    image_size = img.shape
    frame_size = frame.shape
    ROI = [[0,image_size[0]],[0,image_size[1]]]  # the whole image of the object
    if left_corner[1] + image_size[1] > frame_size[1]:
        # outside the right bounding edge
        ROI[1] = [0, frame_size[1] - left_corner[1]]
    if left_corner[1] < 0:
        # outside the left bounding edge
        ROI[1] = [-left_corner[1], image_size[1]]

    if left_corner[0] + image_size[0] > frame_size[0]:
        # outside the bottom bounding edge
        ROI[0] = [0, frame_size[0] - left_corner[0]]
    if left_corner[0] < 0:
        # outside the upper bounding edge
        ROI[0] = [-left_corner[0], image_size[0]]

    return img[ROI[0][0]:ROI[0][1], ROI[1][0]:ROI[1][1]] 

In [7]:
def create_mask(img):
    return np.zeros(img.shape[:2], dtype="uint8")

Orbital Mechanics

In [8]:
def propagate_based_on_vector(r,v, t):
    rf, vf = pykep.propagate_lagrangian(r0 = r, v0 = v, tof = t,mu = 398600)
    return np.array(rf), np.array(vf)

def propagate_based_on_elements(elements, t, kepler=True):
    r, v = pykep.par2ic(elements, mu=398600)
    rf,vf = pykep.propagate_lagrangian(r0 = r, v0 = v, tof = t,mu = 398600)
    if kepler == False:  # return vectors
        return np.array(rf),np.array(vf)
    if kepler == True:  # return kepler elements
        return pykep.ic2par(rf,vf,mu = 398600)

Utility

In [9]:
def delete_white_background(img, write=False):
    """
    Takes the images of Debris that I made in paint.
    Inverts the colours and deletes the white background.
    """
    img = np.invert(img)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    _, alpha = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY)
    
    # Using cv2.split() to split channels 
    # of coloured image
    b, g, r = cv2.split(img)

    # Making list of Red, Green, Blue
    # Channels and alpha
    rgba = [b,g,r, alpha]

    # Using cv2.merge() to merge rgba
    # into a coloured/multi-channeled image
    dst = cv2.merge(rgba,3)
    print("dst shape", dst.shape)

    # Writing and saving to a new image
    if write is True:
        cv2.imwrite("debris/deb.png", dst)
    return dst        

FOV

In [10]:
def get_R(r,v):
    """
    Get the rotation matrix. New y will point in -r direction, new x will point in v direction.
    """
    unit_r = r/np.linalg.norm(r)
    unit_v = v/np.linalg.norm(v)

    R = np.array([[unit_v[0], -unit_r[0], unit_r[1]*unit_v[2] - unit_v[1]*unit_r[2]],
                [unit_v[1], -unit_r[1], unit_v[0]*unit_r[2] - unit_r[0]*unit_v[2]],
                [unit_v[2], -unit_r[2], unit_r[0]*unit_v[1] - unit_v[0]*unit_r[1]]])
    R = R.transpose()
    
    return R

def to_body_axis(vec,r, v):
    """
    Calculates the rotation matrix and applies it. Returns the converted vector.
    r and v are used to calculate the rotation matrix and vector is the vector to be 
    rotated.
    """
    R = get_R(r,v)
    
    return R.dot(vec)

In [11]:
# Checks if a matrix is a valid rotation matrix.
def isRotationMatrix(R) :
    Rt = np.transpose(R)
    shouldBeIdentity = np.dot(Rt, R)
    I = np.identity(3, dtype = R.dtype)
    n = np.linalg.norm(I - shouldBeIdentity)
    return n < 1e-6
 
# Calculates rotation matrix to euler angles
# The result is the same as MATLAB except the order
# of the euler angles ( x and z are swapped ). XYZ
def rotationMatrixToEulerAngles(R) :
 
    assert(isRotationMatrix(R))
 
    sy = math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])
 
    singular = sy < 1e-6
 
    if  not singular :
        x = math.atan2(R[2,1] , R[2,2])
        y = math.atan2(-R[2,0], sy)
        z = math.atan2(R[1,0], R[0,0])
    else :
        x = math.atan2(-R[1,2], R[1,1])
        y = math.atan2(-R[2,0], sy)
        z = 0
 
    return np.array([np.degrees(x), np.degrees(y), np.degrees(z)])

In [253]:
vec = [1,3,0]
r = [1,0,0]
v = [0,1,0]
R = get_R(r,v)
print(R)
print("Euler", rotationMatrixToEulerAngles(R))
R.dot(vec)

[[ 0.  1.  0.]
 [-1. -0. -0.]
 [ 0.  0.  1.]]
Euler [  0.  -0. -90.]


array([ 3., -1.,  0.])

In [12]:
def angle_between(r1,r2):
    """
    Calculate the angle between two vectors
    """
    unit_1 = r1/np.linalg.norm(r1)
    unit_2 = r2/np.linalg.norm(r2)
    return np.arccos(np.dot(unit_1,unit_2))

def behind_earth(r1, r2, R=6371):
    """
    Checks if the target is between the Earth and observer based on 
    a line segment intersecting the sphere.
    Returns true if the target is behind the Earth.
    :param r1: observer
    :param r2: target
    """
    if angle_between(r1,r2) < np.pi/2:  
        return False  # does not intersect
    
    phi = np.arccos(np.dot(r1, r2-r1)/(np.linalg.norm(r1)*np.linalg.norm(r2-r1)))
    H = np.linalg.norm(r1)*np.sin(phi)
    
    if H <= R:
        return True  # intersects
    if H > R:
        return False  # does not intersect

def in_FOV(r1, v1, r2, max_vertical=np.radians(21.3), max_horisontal=np.radians(30)):
    """
    Checks if the observer should be able to see the target.
    1 for observer. 2 for target.
    Returns True if yes
    """

    rel = r2 - r1

    if behind_earth(r1,r2) is True:
        print("failed cause earth")
        return False
    
    rel_unit = (rel)/np.linalg.norm(rel)
    #print("relative in global", rel)
    rel_unit = to_body_axis(rel_unit,r1,v1)
    print("relative in body", rel_unit)
    if rel_unit[1] < 0:
        print("failed cause y")
        return False

    horisontal_angle = abs(np.arctan(rel_unit[0]/rel_unit[1]))  # in the x direction
    vertical_angle = abs(np.arctan(rel_unit[2]/rel_unit[1]))  # in the z direction
    print("angles (hor,ver)", np.degrees(horisontal_angle),np.degrees(vertical_angle))
    if horisontal_angle <= max_horisontal and vertical_angle <= max_vertical:
        return True
    else:
        print("failed cause angles")
        return False

In [10]:
r1 = np.array([7300,0,0])
r2 = np.array([7200,0,0])
behind_earth(r1, r2)

False

Initialisation

In [14]:
elements_sat = [7350,0.0,np.radians(0),np.radians(0),np.radians(0),np.radians(0)]
elements_deb = [7133.8,0.0043,np.radians(10),np.radians(0),np.radians(0),np.radians(0)]

In [35]:
vs = "Earth/test.avi"
img_path = "debris\deb1.png"

In [36]:
object_og = cv2.imread(img_path)
object_og = np.invert(object_og)  # get white object

In [ ]:
object_og = cv2.imread(img_path)
#print(object_og.shape)
#object_og = delete_white_background(object_og, write=False)
#print(object_og.shape)
distance = 7000
object_resized = resize(object_og, (500,500), distance)
cv2.imshow("resized", object_resized)
cv2.waitKey(0)

In [27]:
cap = cv2.VideoCapture(vs)  # open the video file
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter('Earth/Earth_ball.avi', cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 30, (frame_width, frame_height))
t = 0
if cap.isOpened() is False:
    print("Error opening video file")


Body

In [28]:
while cap.isOpened():
    ret0, frame = cap.read()
    print(ret0)
    if ret0 is True:
        rO, vO = propagate_based_on_elements(elements_sat, t, kepler=False)
        rT, vT = propagate_based_on_elements(elements_deb, t, kepler=False)
        t+=1  # move ahead by one second
        print("Observer pos", rO)
        if in_FOV(rO, vO, rT) is True:
            rel = rT - rO
            #print("distance", distance)
            rel = to_body_axis(rel, rO, vO)
            object_resized = resize(object_og, (10,10), rel)  # size of target in pixels
            if object_resized is not False:
                print("relative vector in body axis", rel)
                #print("object size", object_resized.shape)
                left_corner = get_left_corner(rel,object_resized.shape)  # location of target in the frame
                print("left corner", left_corner)
                
                #print("object resized", object_resized.shape)
                roi = roi_in_frame(left_corner, object_resized.shape, frame)
                print("roi shape", roi.shape)
                # create mask and inverse mask
                object_in_frame = obj_in_frame(left_corner,frame,object_resized)
                #cv2.imshow("object in frame", object_in_frame)
                gray = cv2.cvtColor(object_in_frame, cv2.COLOR_BGR2GRAY)
                ret, mask = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)
                mask_inv = cv2.bitwise_not(mask)
                print("mask shape", mask.shape)
                #cv2.imshow("mask", mask)
                #cv2.imshow("inverse mask", mask_inv)
                # black out the area of object in ROI
                frame_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)
                object_fg = cv2.bitwise_and(object_in_frame, object_in_frame, mask=mask)
                #print("object_fg", object_fg.shape)
                #print("frame_bg", frame_bg.shape)
                #cv2.imshow("frame background", frame_bg)
                #cv2.imshow("object foreground", object_fg)
                dst = cv2.add(frame_bg, object_fg)
                #cv2.imshow("dst", dst)
                if left_corner[0] <= 0:
                    frame[0:roi.shape[0],left_corner[1]:roi.shape[1]+left_corner[1]] = dst
                else:
                    frame[left_corner[0]:roi.shape[0]+left_corner[0],left_corner[1]:roi.shape[1]+left_corner[1]] = dst
                #cv2.imshow("Earth with ball", frame)
                cv2.waitKey(0)
        out.write(frame)
    else:
        break

out.release()
cap.release()
cv2.destroyAllWindows()

True
Observer pos [7350.    0.    0.]
relative in body [0. 1. 0.]
angles (hor,ver) 0.0 0.0
relative vector in body axis [  0.      246.87534   0.     ]
location vector in pixels [   0.        1381.2156507    0.       ]
left corner (484, 934)
roi shape (112, 112, 3)
mask shape (112, 112)
True
Observer pos [7349.9963108     7.36418864    0.        ]
relative in body [0.00111906 0.99998543 0.00528034]
angles (hor,ver) 0.06411805637554306 0.3025425789733269
relative vector in body axis [  0.27627132 246.87544801   1.30360438]
location vector in pixels [   1.54567929 1381.2156507     7.29338936]
left corner (477, 936)
roi shape (112, 112, 3)
mask shape (112, 112)
True
Observer pos [7349.98524319   14.72836989    0.        ]
relative in body [0.00223801 0.99994174 0.01056019]
angles (hor,ver) 0.12823587736244768 0.6050671571945385
relative vector in body axis [  0.55254304 246.87577204   2.6072073 ]
location vector in pixels [   3.09135678 1381.2156507    14.58675147]
left corner (470, 938)


KeyboardInterrupt: 

Debug

In [53]:
G_CS = []
B_CS = []
corners = []
for t in range(80):
    rO, vO = propagate_based_on_elements(elements_sat, t, kepler=False)
    rT, vT = propagate_based_on_elements(elements_deb, t, kepler=False)
    print("Observer pos", rO)
    if in_FOV(rO, vO, rT) is True:
        rel = rT - rO
        rel_unit = rel/np.linalg.norm(rel)
        G_CS.append(rel_unit)
        #print("distance", distance)
        rel_unit = to_body_axis(rel_unit, rO, vO)
        rel = to_body_axis(rel, rO, vO)
        B_CS.append(rel_unit)
        object_resized = resize(object_og, (10,10), rel)  # size of target in pixels
        if object_resized is not False:
            print("relative vector in body axis", rel)
            #print("object size", object_resized.shape)
            left_corner = get_left_corner(rel,object_resized.shape)  # location of target in the frame
            print("left corner", left_corner)
            corners.append(left_corner)

Observer pos [7350.    0.    0.]
relative in body [0. 1. 0.]
angles (hor,ver) 0.0 0.0
relative vector in body axis [  0.      246.87534   0.     ]
location vector in pixels [   0.        1381.2156507    0.       ]
left corner (484, 934)
Observer pos [7349.9963108     7.36418864    0.        ]
relative in body [0.00111906 0.99998543 0.00528034]
angles (hor,ver) 0.06411805637554306 0.3025425789733269
relative vector in body axis [  0.27627132 246.87544801   1.30360438]
location vector in pixels [   1.54567929 1381.2156507     7.29338936]
left corner (477, 936)
Observer pos [7349.98524319   14.72836989    0.        ]
relative in body [0.00223801 0.99994174 0.01056019]
angles (hor,ver) 0.12823587736244768 0.6050671571945385
relative vector in body axis [  0.55254304 246.87577204   2.6072073 ]
location vector in pixels [   3.09135678 1381.2156507    14.58675147]
left corner (470, 938)
Observer pos [7349.96679719   22.09253636    0.        ]
relative in body [0.00335677 0.99986892 0.01583908

In [65]:
G_CS

[array([-1.,  0.,  0.]),
 array([-9.99986052e-01,  1.17138965e-04,  5.28033629e-03]),
 array([-9.99944212e-01,  2.34264023e-04,  1.05601914e-02]),
 array([-9.99874492e-01,  3.51361273e-04,  1.58390842e-02]),
 array([-9.99776911e-01,  4.68416824e-04,  2.11165343e-02]),
 array([-9.99651497e-01,  5.85416805e-04,  2.63920617e-02]),
 array([-9.99498285e-01,  7.02347367e-04,  3.16651874e-02]),
 array([-9.99317318e-01,  8.19194689e-04,  3.69354334e-02]),
 array([-9.99108647e-01,  9.35944987e-04,  4.22023232e-02]),
 array([-0.99887233,  0.00105258,  0.04746538]),
 array([-0.99860843,  0.0011691 ,  0.05272414]),
 array([-0.99831703,  0.00128548,  0.05797811]),
 array([-0.9979982 ,  0.0014017 ,  0.06322684]),
 array([-0.99765203,  0.00151776,  0.06846986]),
 array([-0.99727862,  0.00163364,  0.07370671]),
 array([-0.99687808,  0.00174933,  0.07893691]),
 array([-0.99645051,  0.00186482,  0.08416002]),
 array([-0.99599603,  0.00198008,  0.08937557]),
 array([-0.99551476,  0.00209512,  0.09458312]

In [64]:
B_CS

[array([0., 1., 0.]),
 array([0.00111906, 0.99998543, 0.00528034]),
 array([0.00223801, 0.99994174, 0.01056019]),
 array([0.00335677, 0.99986892, 0.01583908]),
 array([0.00447523, 0.99976701, 0.02111653]),
 array([0.0055933 , 0.99963602, 0.02639206]),
 array([0.00671087, 0.999476  , 0.03166519]),
 array([0.00782784, 0.999287  , 0.03693543]),
 array([0.00894413, 0.99906905, 0.04220232]),
 array([0.01005963, 0.99882223, 0.04746538]),
 array([0.01117424, 0.99854659, 0.05272414]),
 array([0.01228786, 0.99824223, 0.05797811]),
 array([0.01340041, 0.99790921, 0.06322684]),
 array([0.01451178, 0.99754764, 0.06846986]),
 array([0.01562188, 0.9971576 , 0.07370671]),
 array([0.01673061, 0.99673921, 0.07893691]),
 array([0.01783788, 0.99629258, 0.08416002]),
 array([0.0189436 , 0.99581783, 0.08937557]),
 array([0.02004766, 0.99531509, 0.09458312]),
 array([0.02114999, 0.99478449, 0.09978222]),
 array([0.02225048, 0.99422619, 0.10497241]),
 array([0.02334905, 0.99364032, 0.11015327]),
 array([0.02

In [54]:
corners[15]

(375, 958)

In [66]:
t=0
new_corners = []
for point in corners:
    new_point = [point[1]+112/2, point[0]+112/2]
    new_point.append(t)
    print(new_point)
    new_corners.append(new_point)
    t+=1

[990.0, 540.0, 0]
[992.0, 533.0, 1]
[994.0, 526.0, 2]
[995.0, 519.0, 3]
[997.0, 511.0, 4]
[998.0, 504.0, 5]
[1000.0, 497.0, 6]
[1001.0, 489.0, 7]
[1003.0, 482.0, 8]
[1004.0, 475.0, 9]
[1006.0, 468.0, 10]
[1008.0, 460.0, 11]
[1009.0, 453.0, 12]
[1011.0, 446.0, 13]
[1012.0, 438.0, 14]
[1014.0, 431.0, 15]
[1015.0, 424.0, 16]
[1017.0, 417.0, 17]
[1018.0, 409.0, 18]
[1020.0, 402.0, 19]
[1021.0, 395.0, 20]
[1023.0, 387.0, 21]
[1025.0, 380.0, 22]
[1026.0, 373.0, 23]
[1028.0, 366.0, 24]
[1029.0, 358.0, 25]
[1031.0, 351.0, 26]
[1032.0, 344.0, 27]
[1034.0, 336.0, 28]
[1035.0, 329.0, 29]
[1037.0, 322.0, 30]
[1038.0, 315.0, 31]
[1040.0, 307.0, 32]
[1041.0, 300.0, 33]
[1043.0, 293.0, 34]
[1045.0, 285.0, 35]
[1046.0, 278.0, 36]
[1048.0, 271.0, 37]
[1049.0, 264.0, 38]
[1051.0, 256.0, 39]
[1052.0, 249.0, 40]
[1054.0, 242.0, 41]
[1055.0, 235.0, 42]
[1057.0, 227.0, 43]
[1058.0, 220.0, 44]
[1060.0, 213.0, 45]
[1062.0, 205.0, 46]
[1063.0, 198.0, 47]
[1065.0, 191.0, 48]
[1066.0, 184.0, 49]
[1068.0, 176.0, 

In [67]:
new_corners

[[990.0, 540.0, 0],
 [992.0, 533.0, 1],
 [994.0, 526.0, 2],
 [995.0, 519.0, 3],
 [997.0, 511.0, 4],
 [998.0, 504.0, 5],
 [1000.0, 497.0, 6],
 [1001.0, 489.0, 7],
 [1003.0, 482.0, 8],
 [1004.0, 475.0, 9],
 [1006.0, 468.0, 10],
 [1008.0, 460.0, 11],
 [1009.0, 453.0, 12],
 [1011.0, 446.0, 13],
 [1012.0, 438.0, 14],
 [1014.0, 431.0, 15],
 [1015.0, 424.0, 16],
 [1017.0, 417.0, 17],
 [1018.0, 409.0, 18],
 [1020.0, 402.0, 19],
 [1021.0, 395.0, 20],
 [1023.0, 387.0, 21],
 [1025.0, 380.0, 22],
 [1026.0, 373.0, 23],
 [1028.0, 366.0, 24],
 [1029.0, 358.0, 25],
 [1031.0, 351.0, 26],
 [1032.0, 344.0, 27],
 [1034.0, 336.0, 28],
 [1035.0, 329.0, 29],
 [1037.0, 322.0, 30],
 [1038.0, 315.0, 31],
 [1040.0, 307.0, 32],
 [1041.0, 300.0, 33],
 [1043.0, 293.0, 34],
 [1045.0, 285.0, 35],
 [1046.0, 278.0, 36],
 [1048.0, 271.0, 37],
 [1049.0, 264.0, 38],
 [1051.0, 256.0, 39],
 [1052.0, 249.0, 40],
 [1054.0, 242.0, 41],
 [1055.0, 235.0, 42],
 [1057.0, 227.0, 43],
 [1058.0, 220.0, 44],
 [1060.0, 213.0, 45],
 [10

In [68]:
%store new_corners

Stored 'new_corners' (list)


In [62]:
df = pd.DataFrame(new_corners)
df.to_csv('data/debug.csv', index=False)